In [11]:
import requests
import pandas as pd
import numpy as np
import csv

endpoint = "https://data.cityofnewyork.us/resource/fhrw-4uyv.json"

def read_zip_population_data():
	# Read in CSV file for zip code population data

	input_file = pd.read_excel("../../downloads/Zips2010.xlsx")
	populations = {}
	for row in input_file:
		key, value = row
		populations[key] = value
	return populations

def group_zips_by_borough():
	endpoint = "https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$where=incident_zip IS NOT NULL"
	response = requests.get(endpoint)
	borough_zips = {}

	# Get ditionary containing zip code population data
	populations = read_zip_population_data()

	for entry in response.json():
		borough = entry['borough']
		inc_zip = entry['incident_zip']

		# If an entry does not yet exist for this borough, make one
		if not borough in borough_zips:
			borough_zips[borough] = {}

		# Get all zip codes stored for this borough
		zips = borough_zips[borough].keys()

		# If no information on zipcode has been stored yet, store it
		if not inc_zip in zips:
			population = 0

			# Get population of zipcode according to 2010 statistics
			if inc_zip in populations.keys():
				population = populations[inc_zip]

			# Save entry in dictionary in format
			# borough_zips { 'borough_name' : { 'zip_code' : 'zip_code_population' } }
			borough_zips[borough][inc_zip] = population
	return borough_zips

def get_all_complain_types():
	endpoint = "https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$where=complaint_type IS NOT NULL"
	response = requests.get(endpoint)
	complaint_types = []
	for entry in response.json():
		complaint_types.append(entry['complaint_type'])
	return complaint_types

def get_entries_by_year(year):
	endpoint = 'https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$where=created_date between "%s-01-01T00:00:00" and "%s-12-31T23:59:59"' % (year, year)
	response = requests.get(endpoint)
	return response.json()

if __name__ == "__main__":
	# Get all complaint types
	complaint_types = get_all_complain_types()

	# Get the number of occurences for each complaint type
	ct_series = pd.Index(complaint_types)

	# Sort the complaint types by descending order
	ct_dict = ct_series.value_counts()

	# Get top ten complaint types and store in array
	top_ten_complaints = ct_dict.keys()[:10]
	print(top_ten_complaints)

	# Get zip codes and population for each borough
	borough_zips = group_zips_by_borough()
	print(borough_zips)

	# Example of getting population for BROOKLYN
	brooklyn_population = 0
	for zip_code in borough_zips['BROOKLYN'].keys():
		brooklyn_population += int(borough_zips['BROOKLYN'][zip_code])
	print(brooklyn_population)

	# COMMENTED PRINT OUT FOR EASY READABILITY
	entries = get_entries_by_year(2017)
	e_df = pd.DataFrame(entries)
	# print(e_df)

Index(['HEAT/HOT WATER', 'Noise - Residential', 'Illegal Parking',
       'Blocked Driveway', 'Request Large Bulky Item Collection', 'PLUMBING',
       'Missed Collection (All Materials)', 'UNSANITARY CONDITION',
       'PAINT/PLASTER', 'Street Condition'],
      dtype='object')


ValueError: too many values to unpack (expected 2)